In [4]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud
import random
%load_ext autoreload
%autoreload 2
from chamferdist import ChamferDistance

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu102_pyt1100/download.html

Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py38_cu102_pyt1100/download.html
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/mohamed/anaconda3/lib/python3.7/site-packages/numpy-1.21.2.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.


In [5]:
num_vertices = 150

In [6]:
config = {
    'experiment_name': 'vertix_edge_training',
    'device': 'cuda:0',  
    'is_overfit': False,
    'batch_size': 8,
    'resume_ckpt': None,
    'learning_rate': 0.0005,
    'max_epochs': 1000,
    'print_every_n': 10,
    'validate_every_n': 4480,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'train',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2
}

In [8]:
train_dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "train", threshold=config["num_vertices"], num_trajectories=config["num_trajectories"])


In [9]:
train_dataset.filter_data()

Length of dataset: 153528
Filtering data ..


100%|██████████████████████████████████| 153528/153528 [03:53<00:00, 658.34it/s]

Length of dataset: 35840


In [ ]:
from training import vertix_edge_train

vertix_edge_train.main(config)

Device: cuda:0
Length of dataset: 153528
Filtering data ..


100%|█████████████████████████████████| 153528/153528 [00:29<00:00, 5178.05it/s]


Length of dataset: 35840


100%|████████████████████████████████| 35840/35840 [00:00<00:00, 1072615.58it/s]


Class 03001627 has 6512 shapes
Class 04530566 has 2680 shapes
Class 04379243 has 8444 shapes
Class 02691156 has 5876 shapes
Class 03636649 has 2056 shapes
Class 02958343 has 3560 shapes
Class 04256520 has 4844 shapes
Class 02933112 has 1868 shapes
Length of dataset: 32304
Filtering data ..


100%|███████████████████████████████████| 32304/32304 [00:06<00:00, 5070.35it/s]


Length of dataset: 7548


100%|██████████████████████████████████| 7548/7548 [00:00<00:00, 1038974.98it/s]


Class 02691156 has 1322 shapes
Class 03001627 has 1296 shapes
Class 04379243 has 1696 shapes
Class 02958343 has 726 shapes
Class 04256520 has 1060 shapes
Class 03636649 has 504 shapes
Class 02933112 has 388 shapes
Class 04530566 has 556 shapes
[000/00009] train_loss: 11.497318, train_vertices_loss: 11.190041, train_edges_loss: 0.307277
[000/00019] train_loss: 7.388511, train_vertices_loss: 7.271842, train_edges_loss: 0.116669
[000/00029] train_loss: 7.230489, train_vertices_loss: 7.120060, train_edges_loss: 0.110429
[000/00039] train_loss: 4.464786, train_vertices_loss: 4.364349, train_edges_loss: 0.100437
[000/00049] train_loss: 3.158190, train_vertices_loss: 3.066189, train_edges_loss: 0.092001
[000/00059] train_loss: 2.694729, train_vertices_loss: 2.596472, train_edges_loss: 0.098257
[000/00069] train_loss: 2.428509, train_vertices_loss: 2.334508, train_edges_loss: 0.094001
[000/00079] train_loss: 1.842119, train_vertices_loss: 1.742235, train_edges_loss: 0.099884
[000/00089] train_

In [ ]:
from inference.inference_vertix import InferenceHandlerVertixModel

# create a handler for inference using a trained checkpoint
inferer = InferenceHandlerVertixModel('runs/vertix_training/model_best.ckpt', num_vertices)

In [111]:
random_sample = random.randint(0,len(train_dataset))

In [112]:
random_input = train_dataset[random_sample]["input_sdf"]
gt_pointcloud = train_dataset[random_sample]["target_vertices"]
num_points = int(train_dataset[random_sample]["input_mask"].sum())

In [113]:
num_points

20

In [115]:
visualize_pointcloud(gt_pointcloud, point_size=0.01)

Output()

In [118]:
output_pointcloud = inferer.infer_single(random_input)

In [120]:
visualize_pointcloud(output_pointcloud, point_size=0.01)

Output()